Load file
======

In [2]:
import pandas as pd
with open('../data/clean_ingredients_new.csv','r') as fp:
    data = pd.read_csv(fp)

In [3]:
data.head()

,recipe_id,ingredients
0,220000,angel food cake mix instant coffee coffee liqu...
1,220001,artichokes roma plum tomatoes round tomatoes t...
2,220002,cabbage salt ground beef ground pork white ric...
3,220003,green bell peppers seeds ground beef white ric...
4,220004,chicken parsnips carrots celery onion dill par...


Get training data ( [recipe_idx, ingredient_idx] )
============== 

In [3]:
import random
import numpy as np

class RecipeInfo():
    
    def __init__(self, data):
        
        self.vocabulary = set()
        self.word_to_index = {}
        
        # bulid the vocabulary
        
        for p in data['ingredients']:
            
            self.vocabulary.update(p.split())
        
        self.vocabulary = list(self.vocabulary)
        
        for i, w in enumerate(self.vocabulary):
            
            self.word_to_index[w] = i
            
        # use word index in ingredients
        
        self.ingredients = data['ingredients'].apply(self.wordstr_to_indexlist)
        
     
    def wordstr_to_indexlist(self, wordstr):
        
        return [self.word_to_index[w] for w in wordstr.split()]
    
    
    def get_samples(self):
        
        samples = self.ingredients.apply(lambda x: random.choice(x))
        
        samples = np.array(samples.reset_index().values, dtype = int)
        
        return samples[:,0], samples[:,1]
        
        

In [4]:
recipes = RecipeInfo(data)

Train recipe2vec
====

In [5]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Embedding, Flatten, Activation, concatenate
from keras.utils import to_categorical


vocabulary = len(recipes.vocabulary)

vector_dim = 100


num_recipes = len(recipes.ingredients)


xrecipe_input = Input(shape=(1,))
xrecipe = Embedding(num_recipes, vector_dim, input_length=1, name='recipes')(xrecipe_input)
x = Flatten()(xrecipe)


x = Dense(vocabulary, activation='softmax')(x)

model = Model(inputs=xrecipe_input, outputs=x)

model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

epoch_num=1

batchsize =128

for i in range(100):

    input_recipe, output_ingredient = recipes.get_samples()
    
    output_ingredient = to_categorical(output_ingredient, vocabulary)
    
    model.fit(input_recipe, output_ingredient, epochs=epoch_num, batch_size=batchsize)
    


Using TensorFlow backend.


Epoch 1/1
17767/17767 [==============================] - 4s 251us/step - loss: 8.2891 - acc: 0.0203
Epoch 1/1
17767/17767 [==============================] - 1s 61us/step - loss: 8.1370 - acc: 0.0324
Epoch 1/1
17767/17767 [==============================] - 1s 61us/step - loss: 7.9603 - acc: 0.0358
Epoch 1/1
17767/17767 [==============================] - 1s 60us/step - loss: 7.6070 - acc: 0.0340
Epoch 1/1
17767/17767 [==============================] - 1s 60us/step - loss: 6.9909 - acc: 0.0342
Epoch 1/1
17767/17767 [==============================] - 1s 61us/step - loss: 6.3506 - acc: 0.0295
Epoch 1/1
17767/17767 [==============================] - 1s 60us/step - loss: 6.0122 - acc: 0.0317
Epoch 1/1
17767/17767 [==============================] - 1s 60us/step - loss: 5.9214 - acc: 0.0338
Epoch 1/1
17767/17767 [==============================] - 1s 60us/step - loss: 5.8907 - acc: 0.0405
Epoch 1/1
17767/17767 [==============================] - 1s 60us/step - loss: 5.8602 - acc: 0.0462
Epoch 1/1

Get recipe2vec
=======

In [6]:
w = model.layers[1].get_weights()[0]

In [7]:
w[:1]

array([[-0.01149134, -0.34488228,  0.3607328 , -0.47375178, -0.07815014,
         0.04152936,  0.0024725 , -0.01838634, -0.23506631, -0.49001384,
        -0.1469217 , -0.07052254, -0.2728073 ,  0.16230299,  0.02332043,
         0.13378231, -0.41966128, -0.01127326, -0.31054604, -0.187339  ,
        -0.21311863, -0.06535441,  0.16378103,  0.12450656, -0.44452214,
         0.23801893,  0.10282075,  0.03085463, -0.05109084, -0.1666557 ,
         0.1692969 ,  0.01470826,  0.07358979, -0.2777261 , -0.01558435,
        -0.25185603,  0.2673106 , -0.00274724, -0.09374077, -0.25507525,
         0.20220809, -0.35605183, -0.09485834,  0.0339432 , -0.02907767,
         0.1594381 ,  0.15831898, -0.07197108, -0.10204753,  0.06906427,
        -0.23589833, -0.19205865,  0.02974932,  0.03229986, -0.07700022,
        -0.2329895 , -0.11151381,  0.06738779,  0.20036733, -0.25502965,
         0.36129823, -0.3490066 , -0.08100104,  0.05960202, -0.23302889,
         0.267132  ,  0.02369494,  0.50340587,  0.3

In [8]:
data['vector'] = list(w)

In [9]:
data.head()

,recipe_id,ingredients,vector
0,220000,angel food cake mix instant coffee coffee liqu...,"[-0.011491336, -0.34488228, 0.3607328, -0.4737..."
1,220001,artichokes roma plum tomatoes round tomatoes t...,"[-0.36830738, -0.0041211634, 0.04224619, -0.10..."
2,220002,cabbage salt ground beef ground pork white ric...,"[-0.14555676, -0.5396178, -0.009736371, -0.263..."
3,220003,green bell peppers seeds ground beef white ric...,"[-0.1709963, -0.40470457, -0.31730914, -0.0535..."
4,220004,chicken parsnips carrots celery onion dill par...,"[-0.07548174, -0.19428793, 0.25274968, -0.0546..."


In [78]:
data.to_csv('recipeid_ingredients_vector.csv')

Find nearest neighbour
========

In [69]:
class RecipeEmbedding:
    
    def __init__(self, data):
        
        self.embedding_matrix = np.array([v for v in data['vector'].values])
        self.ingredients = data['ingredients'].values
        self.recipe_ids = np.array(data['recipe_id'])
        self.recipeid_to_index = {}
        
        for i, r in enumerate(self.recipe_ids):
            
            self.recipeid_to_index[r] = i
    
    def get_vector(self, w):
        """
        w: recipe id
        
        """
        w = self.recipeid_to_index[w]
        w = self.embedding_matrix[w]
        return w
    
    def get_recipe_ids(self, index):

        if isinstance(index, int):
            return self.recipe_ids[index]
        else:
            recipes = [self.recipe_ids[i] for i in index]
            return recipes

    def distance(self, v0, v1):
        v0 = self.get_vector(v0)
        v1 = self.get_vector(v1)
        return np.abs(v0 - v1)
    
    def find_nearest_recipes(self, v, k):
        
        v = self.get_vector(v)
        distances = np.linalg.norm(self.embedding_matrix - v, axis=1)
        return self.get_recipe_ids(np.argsort(distances)[:k])


In [70]:
recipeem = RecipeEmbedding(data)

In [72]:
recipeem.find_nearest_recipes(220000, 5)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:2287: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))


[220000, 257775, 257773, 234004, 222036]

In [75]:
recipeem.ingredients[recipeem.recipeid_to_index[257773]]

'chocolate sandwich cookies with mint cream filling cream cheese confectioners sugar butter vanilla extract butter nut flavor milk instant chocolate pudding mix instant butterscotch pudding mix whipped topping gummy worms'

In [74]:
recipeem.ingredients[recipeem.recipeid_to_index[220000]]

'angel food cake mix instant coffee coffee liqueur cream cheese heavy cream butter instant french vanilla pudding mix cream cheese heavy cream confectioners sugar vanilla extract'

In [76]:
recipeem.ingredients[recipeem.recipeid_to_index[257775]]

'vanilla wafer cookies cream cheese confectioners sugar butter vanilla extract milk instant vanilla pudding mix whipped topping'